In [29]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import numpy as np
import json

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 52.52,
    "longitude": 13.41,
    "start_date": "2024-07-23",
    "end_date": "2024-08-06",
    "hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
response_info = {
    "coordinates": f"{response.Latitude()} {response.Longitude()}",
    "elevation": f"{response.Elevation()}",
    "timezone": f"{response.Timezone()} {response.TimezoneAbbreviation()}",
    "timezone_difference_to_GMT": f"{response.UtcOffsetSeconds()}"
}

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_dates = pd.date_range(
    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
    freq=pd.Timedelta(seconds=hourly.Interval()),
    inclusive="left"
).strftime('%Y-%m-%d %H:%M:%S')

# Create a list of dictionaries for each observation
observations = []
for date, temp in zip(hourly_dates, hourly_temperature_2m):
    observations.append({
        "date": date,
        "temperature_2m": float(temp),
        "latitude": latitude,
        "longitude": longitude,
        "city_name": city_name
    })

# Combine response_info and observations into a single JSON structure
json_response = {
    "response_info": response_info,
    "weather_data": observations
}

# Convert to JSON string
json_response_str = json.dumps(json_response, indent=4)
print(json_response['weather_data'])

NameError: name 'latitude' is not defined